# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure and acess gmaps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created
clean_cities_data = "clean_cities_data.csv"
clean_cities_df = pd.read_csv("output_data/clean_cities_data.csv", encoding="utf-8")
#Also can use: clean_cities_data_df = pd.read_csv(clean_cities_data)
clean_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cabo San Lucas,22.89,-109.91,84.99,78,5,8.05,MX,1599279129
1,Rivadavia,-33.19,-68.46,51.80,40,0,8.05,AR,1599279495
2,Saint-Philippe,-21.36,55.77,64.40,88,75,9.17,RE,1599279000
3,Hilo,19.73,-155.09,82.40,58,90,9.17,US,1599279496
4,Pevek,69.70,170.31,36.16,83,95,1.77,RU,1599278979
...,...,...,...,...,...,...,...,...,...
533,Barra da Estiva,-13.63,-41.33,56.95,91,44,2.77,BR,1599279677
534,Provincia di Imperia,43.97,7.78,71.60,60,0,1.12,IT,1599279677
535,Touros,-5.20,-35.46,71.60,94,75,6.93,BR,1599279395
536,Villavieja,39.85,-0.18,66.99,79,0,3.18,ES,1599279678


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Checking on columns names and types
clean_cities_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [4]:
# Store latitude and longitude in locations
locations = clean_cities_df[["Lat", "Lng"]]

# Store variable humidity
humidity = clean_cities_df["Humidity"]
temperature = clean_cities_df["Max Temp"]
locations.head()


,Lat,Lng
0,22.89,-109.91
1,-33.19,-68.46
2,-21.36,55.77
3,19.73,-155.09
4,69.70,170.31


In [5]:
# Plot Heatmap
figure_layout = {
    'width': '2000px',
    'height': '1000px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure()

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                dissipating=False, max_intensity=300,
                point_radius=3, opacity = 0.60)

markers = gmaps.marker_layer(locations)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
crit_temperature = (clean_cities_df["Max Temp"] < 80) & (clean_cities_df["Max Temp"] > 70)
crit_windSpeed = clean_cities_df["Wind Speed"] < 15
crit_cloudiness = clean_cities_df.Cloudiness == 0
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness 


# Use boolean indexing (True or False) to filter the weather_df dataframe
ideal_weather_df = clean_cities_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,5,Tiznit Province,29.58,-9.50,75.58,54,0,1.45,MA,1599279496
1,28,Najrān,17.49,44.13,71.60,33,0,4.34,SA,1599279273
2,51,Sabzawār,36.21,57.68,75.38,19,0,4.38,IR,1599278936
3,56,Jardim,-21.48,-56.14,74.05,45,0,5.03,BR,1599279280
4,76,Vila Velha,-20.33,-40.29,73.40,73,0,5.82,BR,1599279334
5,122,Viradouro,-20.87,-48.30,73.40,35,0,4.70,BR,1599279535
6,149,Minzhu,43.73,127.34,73.94,63,0,12.12,CN,1599279544
7,157,Buraidah,26.33,43.98,78.80,31,0,1.32,SA,1599279547
8,171,Rancho Palos Verdes,33.74,-118.39,73.00,71,0,8.16,US,1599279552
9,197,Nurota,40.56,65.69,75.20,31,0,13.87,UZ,1599279560


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df["Hotel Name"] = ""

#hotel_df.head()

In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
        "keyword": "hotel",
        #"location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }

#Loop through cities
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]=f"{lat},{lng}"

#Request
    hoteldata = requests.get(base_url, params).json()

#Exception   
    try:
        hotelname= hoteldata["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"]= hotelname
    except:
        hotel_df.loc[index,"Hotel Name"]= np.nan
        
#Drop NaN
hotel_df.dropna(inplace=True)

hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,51,Sabzawār,36.21,57.68,75.38,19,0,4.38,IR,1599278936,Sabzevar Tourism hotel
3,56,Jardim,-21.48,-56.14,74.05,45,0,5.03,BR,1599279280,Pousada Zamora
4,76,Vila Velha,-20.33,-40.29,73.40,73,0,5.82,BR,1599279334,Sheraton Vitoria Hotel
5,122,Viradouro,-20.87,-48.30,73.40,35,0,4.70,BR,1599279535,POUSADA VIRADOURO
6,149,Minzhu,43.73,127.34,73.94,63,0,12.12,CN,1599279544,Hongxing Hotel
7,157,Buraidah,26.33,43.98,78.80,31,0,1.32,SA,1599279547,Mövenpick Hotel Qassim
8,171,Rancho Palos Verdes,33.74,-118.39,73.00,71,0,8.16,US,1599279552,Terranea Resort
9,197,Nurota,40.56,65.69,75.20,31,0,13.87,UZ,1599279560,Guest House Ruslan Nurata & Tours
10,218,Esperance,-33.87,121.90,71.60,56,0,11.41,AU,1599279567,The Jetty Resort
11,219,Solnechnyy,50.72,136.63,75.27,48,0,11.74,RU,1599279568,"Dom Otdykha ""Pastoral'"""


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

#Create a map using coordinates to set markers
marker_locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer for Hotel map
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))